In [ ]:
from haversine import inverse_haversine, Direction
from math import pi
from spf.gps.boundaries import franklin_safe
import numpy as np

In [ ]:
N = 256
orbits = 2
start_point = franklin_safe.mean(axis=0)

from spf.gps.gps_utils import calc_bearing, swap_lat_long
from spf.rf import pi_norm

gt_theta = np.linspace(0, orbits * 2 * pi, N)
long_lat_circle = [
    swap_lat_long(inverse_haversine(swap_lat_long(start_point), 0.05, dir))
    for dir in gt_theta
]

for idx in range(N):
    a = np.deg2rad(calc_bearing(start_point, long_lat_circle[idx]))
    b = gt_theta[idx]
    # print(a, b)
    assert np.isclose(pi_norm(a - b), 0, atol=0.01)

In [ ]:
start_point

In [ ]:
circle[0]

In [ ]:
import math

math.radians(start_point[1]), np.deg2rad(start_point)

In [ ]:
np.hstack([np.random.rand(3, 2)[:, [0]], np.random.rand(3, 2)[:, [1]]])

In [ ]:
math.atan2(0.5, 3), np.arctan2(0.5, 3)

In [ ]:
np.linspace(0, orbits * 2 * pi, N)[32] / pi

In [ ]:
inverse_haversine(start_point, 0.0, 0)

In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import yaml

from spf.data_collector import rx_config_from_receiver_yaml
from spf.dataset.spf_dataset import pi_norm

# V5 data format
from spf.dataset.v4_data import v4rx_2xf64_keys, v4rx_f64_keys, v4rx_new_dataset
from spf.rf import (
    beamformer_given_steering_nomean,
    get_avg_phase,
    pi_norm,
    precompute_steering_vectors,
    speed_of_light,
)
from spf.sdrpluto.sdr_controller import rx_config_from_receiver_yaml
from spf.utils import random_signal_matrix

"""
theta is the angle from array normal to incident
phi is phase difference

delta_distance  = d*sin(theta)
phi = delta_distance * 2pi / lambda = sin(theta)*d*2pi/lambda
theta = arcsin(lambda * phi / (d*2pi))
"""

fake_yaml = """
# The ip of the emitter
# When the emitter is brought online it is verified
# by a receiver that it actually is broadcasting
emitter:
  type: esp32
  motor_channel: 1

# Two receivers each with two antennas
# When a receiver is brought online it performs
# phase calibration using an emitter equidistant from
# both receiver antenna
# The orientation of the receiver is described in 
# multiples of pi
receivers:
  - receiver-uri: fake
    theta-in-pis: -0.25
    antenna-spacing-m: 0.05075 # 50.75 mm 
    nelements: 2
    array-type: linear
    rx-gain-mode: fast_attack
    rx-buffers: 2
    rx-gain: -3
    buffer-size: 524288
    f-intermediate: 100000 #1.0e5
    f-carrier: 2412000000 #2.5e9
    f-sampling: 16000000 # 16.0e6
    bandwidth: 300000 #3.0e5
    motor_channel: 0
  - receiver-uri: fake
    theta-in-pis: 1.25
    antenna-spacing-m: 0.05075 # 50.75 mm 
    nelements: 2
    array-type: linear
    rx-gain-mode: fast_attack
    rx-buffers: 2
    rx-gain: -3
    buffer-size: 524288
    f-intermediate: 100000 #1.0e5
    f-carrier: 2412000000 #2.5e9
    f-sampling: 16000000 # 16.0e6
    bandwidth: 300000 #3.0e5
    motor_channel: 0


n-thetas: 65
n-records-per-receiver: 5
width: 4000
calibration-frames: 10
routine: null
skip_phase_calibration: true
  

data-version: 5
seconds-per-sample: 5.0
"""


def create_fake_dataset_v4(
    yaml_config_str,
    filename,
    orbits=1,
    n=50,
    noise=0.01,
    phi_drift=0.0,
    time_offset=0.0,
    rx_or_tx="rx",
):
    assert rx_or_tx in ["rx", "tx"]
    yaml_fn = f"{filename}.yaml"
    zarr_fn = f"{filename}.zar"
    seg_fn = f"{filename}_segmentation.pkl"
    for fn in [yaml_fn, zarr_fn, seg_fn]:
        if os.path.exists(fn):
            os.remove(fn)
    yaml_config = yaml.safe_load(yaml_config_str)
    yaml_config["n-records-per-receiver"] = n

    with open(f"{filename}.yaml", "w") as outfile:
        yaml.dump(yaml_config, outfile, default_flow_style=False)
    rx_config = rx_config_from_receiver_yaml(yaml_config["receivers"][0])

    _lambda = speed_of_light / rx_config.lo

    m = v4rx_new_dataset(
        filename=f"{filename}.zarr",
        timesteps=yaml_config["n-records-per-receiver"],
        buffer_size=rx_config.buffer_size,
        n_receivers=len(yaml_config["receivers"]),
        chunk_size=512,
        compressor=None,
        config=yaml_config,
    )

    thetas = pi_norm(
        np.linspace(0, 2 * np.pi * orbits, yaml_config["n-records-per-receiver"])
    )

    def theta_to_phi(theta, antenna_spacing_m, _lambda):
        return np.sin(theta) * antenna_spacing_m * 2 * np.pi / _lambda

    def phi_to_theta(phi, antenna_spacing_m, _lambda, limit=False):
        sin_arg = _lambda * phi / (antenna_spacing_m * 2 * np.pi)
        # assert sin_arg.min()>-1
        # assert sin_arg.max()<1
        if limit:
            edge = 1 - 1e-8
            sin_arg = np.clip(sin_arg, a_min=-edge, a_max=edge)
        v = np.arcsin(_lambda * phi / (antenna_spacing_m * 2 * np.pi))
        return v, np.pi - v

    rnd_noise = np.random.randn(thetas.shape[0])

    # signal_matrix = np.vstack([np.exp(1j * phis), np.ones(phis.shape)])

    for receiver_idx in range(2):
        receiver_thetas = (
            thetas - yaml_config["receivers"][receiver_idx]["theta-in-pis"] * np.pi
        )
        phis_nonoise = theta_to_phi(receiver_thetas, rx_config.rx_spacing, _lambda)
        if rx_or_tx == "tx":
            phis_nonoise *= 0
        phis = pi_norm(phis_nonoise + rnd_noise * noise)
        _thetas = phi_to_theta(phis, rx_config.rx_spacing, _lambda, limit=True)

        start_point = franklin_safe.mean(axis=0).reshape(1, 2)

        long_lat_circle = np.array(
            [
                swap_lat_long(
                    inverse_haversine(swap_lat_long(start_point[0]), 0.05, dir)
                )
                for dir in thetas
            ]
        )

        for record_idx in range(yaml_config["n-records-per-receiver"]):
            big_phi = phis[[record_idx], None].repeat(rx_config.buffer_size, axis=1)
            big_phi_with_noise = big_phi + np.random.randn(*big_phi.shape) * noise
            offsets = np.random.uniform(-np.pi, np.pi, big_phi.shape) * 0
            signal_matrix = (
                np.vstack(
                    [
                        np.exp(
                            1j
                            * (
                                offsets
                                + phi_drift * np.pi * (1 if receiver_idx == 0 else -1)
                            )
                        ),
                        np.exp(1j * (offsets + big_phi_with_noise)),
                    ]
                )
                * 200
            )
            noise_matrix = random_signal_matrix(
                signal_matrix.reshape(-1).shape[0]
            ).reshape(signal_matrix.shape)
            # add stripes
            window_size = 2048 * 4
            for x in range(0, rx_config.buffer_size, window_size):
                if (x // window_size) % 3 == 0:
                    signal_matrix[:, x : x + window_size] = noise_matrix[
                        :, x : x + window_size
                    ]

            data = {
                "gps_timestamp": record_idx * 5.0 + time_offset,
                "gps_lat": (
                    long_lat_circle[record_idx][1]
                    if rx_or_tx == "tx"
                    else start_point[0, 1]
                ),
                "gps_long": (
                    long_lat_circle[record_idx][0]
                    if rx_or_tx == "tx"
                    else start_point[0, 0]
                ),
                "heading": (
                    (calc_bearing(start_point, long_lat_circle[[record_idx]]) + 90)
                    % 360
                    if rx_or_tx == "tx"
                    else 0
                ),
                "system_timestamp": record_idx * 5.0,
                "rx_theta_in_pis": yaml_config["receivers"][receiver_idx][
                    "theta-in-pis"
                ],
                "rx_spacing": rx_config.rx_spacing,
                "rx_lo": rx_config.lo,
                "rx_bandwidth": rx_config.rf_bandwidth,
                "avg_phase_diff": get_avg_phase(signal_matrix),
                "rssis": [0, 0],
                "gains": [0, 0],
            }

            z = m[f"receivers/r{receiver_idx}"]
            z.signal_matrix[record_idx] = signal_matrix
            for k in v4rx_f64_keys + v4rx_2xf64_keys:
                z[k][record_idx] = data[k]
            # nthetas = 64 + 1

            # steering_vectors = precompute_steering_vectors(
            #     receiver_positions=rx_config.rx_pos,
            #     carrier_frequency=rx_config.lo,
            #     spacing=nthetas,
            # )
            # beam_sds = beamformer_given_steering_nomean(
            #     steering_vectors=steering_vectors,
            #     signal_matrix=signal_matrix,
            # )

In [ ]:
create_fake_dataset_v4(fake_yaml, "./test_rx", rx_or_tx="rx")

In [ ]:
create_fake_dataset_v4(fake_yaml, "./test_tx", rx_or_tx="tx")

In [ ]:
from spf.utils import zarr_open_from_lmdb_store


def z_to_long_lat_gps(z):
    return np.vstack([z["receivers/r0/gps_long"], z["receivers/r0/gps_lat"]]).T


z_rx = zarr_open_from_lmdb_store("./test_rx.zarr")
z_tx = zarr_open_from_lmdb_store("./test_tx.zarr")

In [ ]:
z_to_long_lat_gps(z_rx).shape

In [ ]:
z_to_long_lat_gps(z_rx)

In [ ]:
import matplotlib.pyplot as plt

# phase diff should be off because of array orientation
plt.plot(z_rx["receivers/r0"]["avg_phase_diff"][:, 0], label="rx_phase")
plt.plot(
    np.deg2rad(calc_bearing(z_to_long_lat_gps(z_rx), z_to_long_lat_gps(z_tx))),
    label="calculated bearing",
)
plt.legend()

In [ ]:
z_tx["receivers/r0"]["avg_phase_diff"][:, 0]

In [ ]:
z["receivers/r0"]["system_timestamp"][:1000]
# 1719364172.076753 gps timestamp

In [ ]:
import time

time.time()

In [ ]:
time.time()

In [ ]:
z.tree()